In [1]:
from envs.train_env import Electric_Car
from models.baseline import BaselineModel
from models.baseline2 import BaselineModel2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from models.tabular_qlearning import TabularQLearning

from envs.feature_engineering import DataHelper

In [2]:
dh = DataHelper()

def learn(env, model):
    states, rewards, infos = [], [], []
    truncated = False
    terminated = False
    while(terminated == False and truncated == False):
        action = model.predict(env)
        obs, reward, termination, truncation, info = env.step(action)
        new_obs = dh.process_data(obs)

        states.append(new_obs)
        rewards.append(reward)
        terminated = termination
        truncated = truncation
        infos.append(info)

    return states, rewards, terminated, truncated, infos

In [3]:
data_path = 'data/train.xlsx'
simulations = 10
learning_rate = 0.5
model = TabularQLearning(data_path)
model.train(simulations, learning_rate)

bins_actions [-1.         -0.89473684 -0.78947368 -0.68421053 -0.57894737 -0.47368421
 -0.36842105 -0.26315789 -0.15789474 -0.05263158  0.05263158  0.15789474
  0.26315789  0.36842105  0.47368421  0.57894737  0.68421053  0.78947368
  0.89473684  1.        ]
bins_battery [ 0.          2.63157895  5.26315789  7.89473684 10.52631579 13.15789474
 15.78947368 18.42105263 21.05263158 23.68421053 26.31578947 28.94736842
 31.57894737 34.21052632 36.84210526 39.47368421 42.10526316 44.73684211
 47.36842105 50.        ]
bins_price [   0.          131.57894737  263.15789474  394.73684211  526.31578947
  657.89473684  789.47368421  921.05263158 1052.63157895 1184.21052632
 1315.78947368 1447.36842105 1578.94736842 1710.52631579 1842.10526316
 1973.68421053 2105.26315789 2236.84210526 2368.42105263 2500.        ]
state before discr [2.500e+01 2.431e+01 1.000e+00 0.000e+00 1.000e+00 1.000e+00 2.007e+03
 1.000e+00]
whole state [   9    0    1    0    1    1 2007    1]
part of state [9 0 1 0 1]
shape 

IndexError: index 19 is out of bounds for axis 0 with size 19